# Projeto 2 - Transfer Learning
Matheus Arataque Uema - 10276949

Dhyogo Nunes Costa - 13096109


---


Este projeto envolve usar a rede MobileNetV2, treinada com a base Imagenet e adapta--la para reconhecer dados da base "cats and dogs".

##  Etapa 1 - Base de dados
Será utilizada a base "cats and dogs" , disponível em: https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip. Observe que o dataset já vem dividido em treino e teste

In [ ]:
train_dir = 'cats_and_dogs_filtered/cats_and_dogs_filtered/train'
validation_dir = 'cats_and_dogs_filtered/cats_and_dogs_filtered/validation'

## Etapa 2 - Redimensionamento
Os dados serão redimensionados para o tamanho 160x160, utilizando batches de tamanho 32 e com seus valores escalados para o intervalo 0 e 1.
Para simplificar esse processo, a classe `tf.keras.preprocessing.image.ImageDataGenerator` (com parâmetro rescale), e seu `método flow_from_directory` (com parâmetros `validation_dir, target_size, batch_size e class_mode`) serão utilizados.

In [ ]:
import tensorflow as tf

# Configuração do ImageDataGenerator para reescalonar as imagens
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

# Carregar as imagens a partir do diretório com redimensionamento e batch size definidos
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(160, 160),  # Redimensiona as imagens para 160x160
    batch_size=32,           # Define o tamanho do batch como 32
    class_mode='binary'      # Configura a classificação binária para gatos e cachorros
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(160, 160),  # Redimensiona as imagens para 160x160
    batch_size=32,           # Define o tamanho do batch como 32
    class_mode='binary'      # Configura a classificação binária para gatos e cachorros
)

## Etapa 3 - Arquitetura da rede MobileNetV2
A MobileNetV2 foi arquitetada visando leveza e eficiência. Para isso, ela minimiza o número de operações e parâmetros através de convoluções de profundidade separável e blocos invertidos residuais. Sua arquitetura pode ser definida da seguinte forma:

**Primeira camada de convolução**: possui 32 filtros e stride de 2, seguida de uma camada de normalização Batch Normalization e ativação ReLU6.

**Blocos invertidos residuais**: consistem em uma convolução 1x1 para expansão, uma consolução depthwise separável (3x3 com stride de 1 ou 2), uma convolução 1x1 para compressão, resíduo entre entrada e saída e a maioria utiliza ReLU6 como ativação.

**Blocos de Expansão e Projeção**: possui uma fase de expansão - aumentando a representação dimensional - onde o número de filtros aumenta e uma de projeção - reduzindo a redundância - onde o número de filtros reduz.

**Bloco Final de Convolução**: camada de convolução 1x1 com 1280 filtros.

**Pooling Global**:  reduz a dimensão espacial para 1x1

**Camada de Saída**: camada totalmente conectada que utiliza softmax para classificação com múltiplas classes e sigmoid para classificação binária

Essa arquitetura reduz o custo computacional através das convoluções empregadas. Além disso, como a maioria das camadas é configurável com fatores de `width multiplier` e `resolution multiplier`, são permitidos ajustes no tamanho da rede e na resolução da entrada, otimizando seu uso. Por conta disso, a MobileNetV2 é altamente precisa para tarefas de visão, como classificação de imagens e detecção de objetos.

Será utilizado o modelo MobileNetV2 com os pesos do ImageNet e sem as camadas totalmente conectadas. Seus pesos serão congelados, o que pode ser verificado no código abaixo.



In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2

# Carregar o modelo MobileNetV2 com pesos do ImageNet e sem as camadas totalmente conectadas
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(160, 160, 3))

# Congelar os pesos do modelo
base_model.trainable = False

# Verificar a estrutura do modelo e confirmar que as camadas estão congeladas
# base_model.summary()

As camadas da base de dados serão adicionadas através de uma camada totalmente conectada. Para fins de simplificação, uma camada `GlobalAveragePooling2D` será utilizada no treino, o qual utilizará como métrica a acurácia.

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Adicionar novas camadas no topo do modelo
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduz a dimensão espacial
x = Dense(1, activation='sigmoid')(x)  # Camada de saída para classificação binária

# Criar o modelo final
model = Model(inputs=base_model.input, outputs=x)

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

## Etapa 4 - Validação

Serão utilizadas três imagens de gatos e três imagens de cachorros para validação do modelo criado.


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Função para pré-processar uma imagem e fazer a previsão
def preprocess_and_predict(img_path, model):
    img = image.load_img(img_path, target_size=(160, 160))  # Carregar e redimensionar a imagem
    img_array = image.img_to_array(img) / 255.0             # Converter para array e escalar
    img_array = np.expand_dims(img_array, axis=0)           # Expandir a dimensão para batch
    prediction = model.predict(img_array)                   # Fazer a previsão
    return prediction[0][0]                                 # Retorna a previsão

# Caminhos para as imagens de gatos e cachorros
image_paths = [
    'path/to/cat1.jpg',
    'path/to/cat2.jpg',
    'path/to/cat3.jpg',
    'path/to/dog1.jpg',
    'path/to/dog2.jpg',
    'path/to/dog3.jpg'
]

# Realizar a previsão e exibir o resultado
for img_path in image_paths:
    pred = preprocess_and_predict(img_path, model)
    label = "Dog" if pred > 0.5 else "Cat"  # Limite de 0.5 para classificação binária
    print(f"Image: {img_path} - Predicted Label: {label} - Confidence: {pred}")


```1/1 [==============================] - 1s 833ms/step
Image: cat1.jpg - Predicted Label: Cat - Confidence: 0.000742390810046345
1/1 [==============================] - 0s 34ms/step
Image: cat2.jpg - Predicted Label: Cat - Confidence: 0.0006298071821220219
1/1 [==============================] - 0s 34ms/step
Image: cat3.jpg - Predicted Label: Dog - Confidence: 0.7990727424621582
1/1 [==============================] - 0s 34ms/step
Image: dog1.jpg - Predicted Label: Dog - Confidence: 0.8726528882980347
1/1 [==============================] - 0s 34ms/step
Image: dog2.jpg - Predicted Label: Dog - Confidence: 0.808917760848999
1/1 [==============================] - 0s 34ms/step
Image: dog3.jpg - Predicted Label: Dog - Confidence: 0.9711649417877197```

Foram testados para cada animal, gato e cachorro, um real, um gerado por IA e um em estilo desenhado.
 As imagens reais foram corretamente identificadas.
 O mesmo pôde ser observado para as imagens geradas em IA. 
 No entanto, para os estilos desenhados, o modelo gerado errou na identificação da imagem de gato utilizado.